In [91]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import tqdm
import sys

In [65]:
def read_csv(directory, folder, filename):
    df = pd.read_csv(directory + folder + filename)
    df = df.replace(r'^\s*$', np.nan, regex=True) #replaces empty strings spacess with NaN
    return df

In [107]:
directory = '/Volumes/GoogleDrive-114506167238584938104/My Drive/common_test_files/'
folder = 'bug_exp/'

df = read_csv(directory, folder, 'train-hrt-30%-NaN.csv')



In [135]:
valid_data_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']

valid = df.select_dtypes(include=valid_data_types)

numeric_one_hot_encoder_columns = []

for col in valid.columns:

    unique = valid[col].unique()
    unique = unique[~np.isnan(unique)] #remove nan
    unique_int_len = len(list(filter(lambda x: x.is_integer(), unique))) #checks to ensure values are ints even if cast as float

    if len(unique) == unique_int_len and (unique_int_len > 2 and unique_int_len < 6): #set 
        numeric_one_hot_encoder_columns.append(col)

print(numeric_one_hot_encoder_columns)


restecg
    

['restecg', 'slope', 'ca', 'thal']


In [139]:
valid['thal'].unique()

array([nan,  2.,  3.,  0.,  1.])

In [67]:
X = df

col_is_binary = []

for col in X.columns:
    feature = X[col]
    
    if feature.isna().sum() > 0: #ensure has nan values
        values = feature.dropna().unique()
        values.sort()
        if list(values) == [0,1]:
            col_is_binary.append(col)
            
        

In [55]:
print(col_is_binary)

['sex', 'exang']


In [68]:
def generate_dataset(rows, columns, percent_missing):

    size = rows * columns
    goal = round(size * percent_missing / 100)
    df = pd.DataFrame(np.random.randint(0,100,size=(rows, columns)))
    nan_count = 0
    
    while nan_count < goal:
        r = np.random.randint(0,rows - 1)
        c = np.random.randint(0,columns - 1) 
        df.iat[r,c]=None
        
        nan_count = df.isna().sum().sum()
    
    return df

In [90]:
df = generate_dataset(100,100,10)



#result = 



In [96]:
imp_mean = IterativeImputer(random_state=0, verbose=2)

imp_mean.fit_transform(df)
# result =  pd.DataFrame(imp_mean.transform(df),columns=df.columns)



def test(val):
    print(val)
print(sys.stdout)

[IterativeImputer] Completing matrix with shape (100, 100)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.38
[IterativeImputer] Change: 48.7600451717533, scaled tolerance: 0.099 
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.74
[IterativeImputer] Change: 5.809198350050295, scaled tolerance: 0.099 
[IterativeImputer] Ending imputation round 3/10, elapsed time 4.16
[IterativeImputer] Change: 0.44923324067432446, scaled tolerance: 0.099 
[IterativeImputer] Ending imputation round 4/10, elapsed time 5.46
[IterativeImputer] Change: 0.017835959607644725, scaled tolerance: 0.099 
[IterativeImputer] Early stopping criterion reached.


In [74]:
result

,0,1,2,3,4,5,6,7,8,9
0,63.0,54.0,56.000000,53.0,54.744257,86.000000,31.000000,81.000000,11.0,57.0
1,98.0,2.0,47.000000,79.0,33.000000,52.704985,24.000000,10.000000,20.0,62.0
2,13.0,59.0,27.000000,68.0,61.000000,52.680326,40.000000,92.000000,20.0,18.0
3,61.0,89.0,50.448151,64.0,0.000000,32.000000,49.181197,19.000000,4.0,2.0
4,21.0,30.0,50.497895,85.0,39.000000,86.000000,3.000000,30.000000,72.0,62.0
...,...,...,...,...,...,...,...,...,...,...
95,36.0,37.0,74.000000,2.0,26.000000,72.000000,49.140118,52.548476,29.0,0.0
96,9.0,99.0,10.000000,0.0,92.000000,69.000000,15.000000,13.000000,3.0,14.0
97,13.0,72.0,78.000000,2.0,6.000000,28.000000,49.159768,45.000000,66.0,16.0
98,55.0,92.0,32.000000,0.0,72.000000,49.000000,43.000000,53.000000,47.0,44.0
